In [1]:
%%configure -f
{
    "conf": {
        "spark.jars": "s3://sjet-datamart-bucket/Jars/mssql-jdbc-6.1.0.jre8.jar"        
    }
}

ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
14,application_1664514829412_0141,pyspark,busy,Link,Link,
17,application_1664514829412_0144,pyspark,dead,Link,Link,
18,application_1664514829412_0147,pyspark,dead,Link,Link,
19,application_1664514829412_0149,pyspark,idle,Link,Link,


In [75]:
UNION_DF = spark.sql('''
                        
                        SELECT * FROM precogdb.df6_red_pitch_attitude_high_approach
                        UNION
                        SELECT * FROM precogdb.df6_amber_pitch_attitude_high_approach
                        UNION
                        SELECT * FROM precogdb.df6_green_pitch_attitude_high_approach
                        
                        ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [76]:
UNION_DF.createOrReplaceTempView('UNION_DF')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [77]:
HRDDF=spark.sql('''SELECT 
                            CAST(ud.exceedanceid AS INT),
                            ud.exceedance,
                            CAST(ud.phaseid AS INT),
                            ud.phase,
                            ud.ecn,
                            ud.employeename,
                            ud.rank,
                            ud.tailno,
                            ud.departure,
                            ud.arrival,
                            ud.flightno AS FlightNumber,
                            ud.date as FLightDate ,
                            ud.chocksoff,
                            ud.chockson,
                            ud.domint,
                            CAST(ud.manufacturerid AS INT),
                            ud.manufacturer,
                            CAST(ud.dataframeid AS INT),
                            ud.dataframe,
                            ud.aircrafttype,
                            ud.aircraftmodel,
                            ud.payload,
                            ad.blocktime as `Block time(Hrs.)`,
                            occurance,
                            flag as type,
                            CAST(trigger_hours AS DECIMAL(8,2)) AS `Trigger Hours`,
                            CAST(occurence_hours AS DECIMAL(8,2)) AS `Occurence Hours`,
                            CAST(duration_min AS DECIMAL(6,2)) AS  `Duration(Min)`,
                            '' AS `CAPT PITCH ATT`,
                            '' AS `CAPT DISPLAY PITCH ATT`,
                            `Pitch-Angle` AS `PITCH ANGLE`,
                            '' AS `Pitch Attitude`,
                            '' AS `RADIO HEIGHT (combined)`,
                            '' AS `RADIO HEIGHT`,
                            '' AS `RADIO ALTITUDE`,
                            `Rad-Alt` AS `RAD ALT`, 
                            '' AS `LEFT MAIN GEAR AIR/GND`,
                            '' AS `LT MAIN GEAR AIR/GND`,
                            '' AS `RIGHT MAIN GEAR AIR/GND`,
                            '' AS `RT MAIN GEAR AIR/GND`,
                            '' AS `MAIN WOW`,
                            `WOW-MLG` AS `WOW MLG`,
                            loadeddate AS LoadDate,
                            filename
                    FROM UNION_DF ud
                    INNER JOIN PilotInfoDF ad ON ud.Flightno=ad.DesigFlightNumber
                                                        AND ud.Date=ad.FlightDate
                                                        AND ud.Departure=ad.FromCity
                                                        AND ud.Arrival=ad.Tocity
                                                        AND ud.ECN=ad.ECN


''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
DAGSDF.createOrReplaceTempView('DAGSDF')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
PilotInfoDF = spark.read.csv('s3://sjet-datamart-bucket/precog/pilot-info/', header=True)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
PilotInfoDF.createOrReplaceTempView('PilotInfoDF')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [8]:
JoinDF = spark.sql('''
                        SELECT 
                            AD1.exceedanceid,
                            AD1.exceedance,
                            AD1.phaseid,
                            AD1.phase,
                            AD1.ecn,
                            AD1.employeename,
                            AD1.rank,
                            AD1.tailno,
                            AD1.departure,
                            AD1.arrival,
                            AD1.flightnumber,
                            AD1.FLightDate ,
                            AD1.chocksoff,
                            AD1.chockson,
                            AD1.domint,
                            AD1.manufacturerid,
                            AD1.manufacturer,
                            AD1.dataframeid,
                            AD1.dataframe,
                            AD1.aircrafttype,
                            AD1.aircraftmodel,
                            AD1.payload,
                            PDF.BlockTime AS `Block time(Hrs.)`,
                            AD1.occurance,
                            AD1.type,
                            AD1.`Trigger Start Hours`,
                            AD1.`Trigger End Hours`,
                            AD1.`Occurence Start Hours`,
                            AD1.`Occurence End Hours`,
                            AD1.`Duration(Min)`,
                            AD1.`G/S ENGAGE`,
                            AD1.`AFCS GLIDESLOPE ACTIVE`,
                            AD1.`AFCS GS Mode`,
                            AD1.`GLIDESLOPE DEVIATION - L`,
                            AD1.`GLIDESLOPE DEV/ELEVATION-L (Dots)`,
                            AD1.`GLIDE PATH DEV PILOT +=ABOVE`,
                            AD1.`ILS GS DEV #1`,
                            AD1.`GLIDESLOPE DEVIATION - R`,
                            AD1.`GLIDESLOPE DEV/ELEVATION-R (Dots)`,
                            AD1.`GLIDE PATH DEV COPILOT +=ABOVE`,
                            AD1.`ILS GS DEV #2`,
                            AD1.`CAPT G/S DEV`, 
                            AD1.`LEFT MAIN GEAR AIR/GND`,
                            AD1.`LT MAIN GEAR AIR/GND`,
                            AD1.`RIGHT MAIN GEAR AIR/GND`,
                            AD1.`RT MAIN GEAR AIR/GND`,
                            AD1.`MAIN WOW`,
                            AD1.`WOW MLG`,
                            AD1.`RADIO HEIGHT (combined)`,
                            AD1.`RADIO HEIGHT`,
                            AD1.`RADIO ALTITUDE`,
                            AD1.`RAD ALT`,
                            AD1.loaddate,
                            AD1.filename
                            
                            FROM DAGSDF AD1
                            JOIN PilotInfoDF PDF 
                            ON AD1.ecn = PDF.ecn
                            AND AD1.departure = PDF.FromCity
                            AND AD1.arrival = PDF.ToCity
                            AND AD1.Flightnumber = PDF.DesigFlightNumber
                            AND AD1.FlightDate = PDF.FlightDate

                    ''')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [78]:
HRDDF \
  .write \
  .format("jdbc") \
  .mode("append") \
  .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
  .option("url", "jdbc:sqlserver://10.150.72.22:1433;databaseName=precog") \
  .option("dbtable", "PitchAttitudeHigh") \
  .option("user", "dmbigdata") \
  .option("password", "$p!Ce@bigData$db") \
  .save()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…